# Import Modules and Inputs

In [20]:
# install geemap package (https://geemap.org/)
    #conda install geemap -c conda-forge

In [21]:
import ee, geemap, time

Intialize Geemap

In [22]:
Map = geemap.Map(center=(10.41, 37.62), zoom=11)

jedeb = ee.FeatureCollection("projects/ee-yilkalg3/assets/Jedeb_Watershed")
#Map.addLayer(jedeb, {}, 'Jedeb Watershed')

Set Dates for filtering images in the irrigation season 

* Dates are based on planting and harvesting date in the area)

In [23]:
start = '2021-12-01'
end = '2022-04-30'
season = ee.Filter.date(start,end)

Import Training Dataset

* Training and validation are randomly selected using random selection tool. 80% are set to be training datasets are the rest 20% set to validation. 

In [24]:
points = ee.FeatureCollection("projects/ee-yilkalg3/assets/TrainingJ")
print(points.size().getInfo())

896


Import Validation Dataset

In [25]:
validationGcp = ee.FeatureCollection("projects/ee-yilkalg3/assets/ValidationJ")
print(validationGcp.size().getInfo())

225


In [26]:
#validationGcp.first().getInfo()

# Sentinel 2 only

In [27]:
# Load Median Sentinel 2 Reflectance 
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Filter the first Sentinel 2 image of the season
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterBounds(jedeb)\
    .filter(season)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .map(maskS2clouds)
 
# Median image
s2median = ee.Image(sentinel2.median()).clip(jedeb)
s2median.bandNames().getInfo()

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

Map.addLayer(s2median, visualization, 'RGB')

## Train the classifier

In [28]:
# Use these bands for prediction (including red edge bands).
bands = ['B2', 'B3', 'B4','B5', 'B6','B7','B8', 'B8A', 'B11', 'B12']

# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = s2median.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

classifier = ee.Classifier.smileRandomForest(128).train(sample, label, bands)

## Classify the Image

In [29]:
# Classify the image with the same bands used for training 
classified_RF1 = s2median.select(bands).classify(classifier)

## Accuracy assessment

Training Accuracy

In [30]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [31]:
#train_accuracy.accuracy().getInfo()

In [32]:
#train_accuracy.kappa().getInfo()

In [33]:
#train_accuracy.producersAccuracy().getInfo()

In [34]:
#train_accuracy.consumersAccuracy().getInfo()

Validation Accuracy

In [35]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF1.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [36]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

In [37]:
test_accuracy.accuracy().getInfo()

0.8133333333333334

In [38]:
test_accuracy.kappa().getInfo()

0.7607776624560161

In [39]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.8142857142857143],
 [0.6176470588235294],
 [0.8181818181818182],
 [0.8867924528301887],
 [1],
 [0.7777777777777778]]

# Sentinel 1 and Sentinel 2

In [40]:
# Import Sentinel-1 collection
sentinel1 =  ee.ImageCollection('COPERNICUS/S1_GRD')\
                .filterBounds(jedeb)\
                .filter(season)\
                .filter(ee.Filter.eq('instrumentMode', 'IW'))\
                .filter(ee.Filter.eq('resolution_meters', 10)) 
                
# Number of tiles in the season
sentinel1 = sentinel1.sort('system:time_start')
sentinel1.size().getInfo()

13

In [41]:
# the acquisition times in the collection, formatted with Python's time module
acq_times = sentinel1.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['12/02/21',
 '12/14/21',
 '12/26/21',
 '01/07/22',
 '01/19/22',
 '01/31/22',
 '02/12/22',
 '02/24/22',
 '03/08/22',
 '03/20/22',
 '04/01/22',
 '04/13/22',
 '04/25/22']

In [42]:
# Apply reduce the radar speckle by smoothing  
smoothing_radius = 50

In [43]:
# using monthly mean backscatter 

s1Dec = ee.Image(sentinel1.filterDate("2021-12-01","2021-12-31").mean()).focal_mean(smoothing_radius, 'circle', 'meters')\
                 .select(["VV","VH"],["VVdec","VHdec"])
s1Jan = ee.Image(sentinel1.filterDate("2022-01-01","2022-01-31").mean()).focal_mean(smoothing_radius, 'circle', 'meters')\
                .select(["VV","VH"],["VVjan","VHjan"]);
s1Feb = ee.Image(sentinel1.filterDate("2022-02-01","2022-02-28").mean()).focal_mean(smoothing_radius, 'circle', 'meters')\
                .select(["VV","VH"],["VVfeb","VHfeb"])
s1Mar = ee.Image(sentinel1.filterDate("2022-03-01","2022-03-31").mean()).focal_mean(smoothing_radius, 'circle', 'meters')\
                .select(["VV","VH"],["VVmar","VHmar"])
s1Apr = ee.Image(sentinel1.filterDate("2022-04-01","2022-04-30").mean()).focal_mean(smoothing_radius, 'circle', 'meters')\
                .select(["VV","VH"],["VVapr","VHapr"])

sentinel1mean = s1Dec.addBands(s1Jan)\
                 .addBands(s1Feb)\
                 .addBands(s1Mar)\
                 .addBands(s1Apr)

#print(sentinel1mean.getInfo())

In [44]:
composite = ee.Image.cat([(s2median),(sentinel1mean)])
sentinel = composite.clip(jedeb)
#print(Sentinel.getInfo())

## Train the Classifier

In [45]:
bands = ['B2', 'B3', 'B4','B8', 'B8A', 'B11', 'B12', 'VVdec', 'VHdec', 'VVjan', 'VHjan', 'VVfeb', 'VHfeb', 'VVmar', 'VHmar', 'VVapr', 'VHapr']
# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = sentinel.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

In [46]:
classifier = ee.Classifier.smileRandomForest(128).train(sample, label, bands)

## Classify the Image

In [47]:
# Classify the image with the same bands used for training.
classified_RF2 = sentinel.select(bands).classify(classifier)

## Accuracy assessment

Training Accuracy

In [48]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [49]:
#train_accuracy.accuracy().getInfo()

In [50]:
#train_accuracy.kappa().getInfo()

In [51]:
#train_accuracy.producersAccuracy().getInfo()

In [52]:
#train_accuracy.consumersAccuracy().getInfo()

Validation Accuarcy

In [53]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF2.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [54]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

In [55]:
test_accuracy.accuracy().getInfo()

0.88

In [56]:
test_accuracy.kappa().getInfo()

0.8457103672474222

In [57]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.9],
 [0.7058823529411765],
 [0.9318181818181818],
 [0.9056603773584906],
 [1],
 [0.7777777777777778]]

In [58]:
test_accuracy.consumersAccuracy().getInfo()

[[0, 1, 0.9, 1, 0.8913043478260869, 0.7741935483870968, 1, 0.875]]

# Improving Classification 

## Vegetation Indices 

EVI

In [59]:
def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

evi = sentinel2.map(getEVI).median().clip(jedeb).select('EVI')

print(evi.getInfo())

{'type': 'Image', 'bands': [{'id': 'EVI', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


NDVI

In [60]:
# Calculate NDVI
def getNDVI(image):
    # Compute the EVI using an expression.
    NDVI = image.expression(
        '((NIR - RED) / (NIR + RED ))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
        }).rename("NDVI")

    image = image.addBands(NDVI)

    return(image)

ndvi = sentinel2.map(getNDVI).median().clip(jedeb).select('NDVI')

print(ndvi.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


NDBI

In [61]:
# Calculate NDBI
def getNDBI(image):
    # Compute the EVI using an expression.
    NDBI = image.expression(
        '((SWIR - NIR) / (SWIR + NIR ))', {
            'SWIR': image.select('B11'),
            'NIR': image.select('B8'),
        }).rename("NDBI")

    image = image.addBands(NDBI)

    return(image)

ndbi = sentinel2.map(getNDBI).median().clip(jedeb).select('NDBI')

print(ndbi.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDBI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


Slope

## Object based Clustering

In [63]:
dem = ee.Image("NASA/NASADEM_HGT/001").select('elevation')

In [85]:
slope = ee.Terrain.slope(dem)
#Map.addLayer(slope, {min: 0, max: 89.99}, 'Slope')
print(slope.getInfo())

{'type': 'Image', 'bands': [{'id': 'slope', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 90}, 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -179.0001388888889, 0, -0.0002777777777777778, 61.00013888888889]}], 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-144.73828125306034, -56.000278238368686], [-105.23242185112113, -56.00027824764527], [-37.40820312968272, -56.00027826661891], [19.752929728747688, -56.00027822905273], [77.08886723064317, -56.00027829591546], [142.6406250064462, -56.000278285607], [179.00049757955398, -56.000277936790376], [179.00057392307653, 61.000277909079024], [138.62011709218748, 61.000278275388574], [90.02441401141857, 61.000278237489134], [18.17968756388447, 61.00027824817684], [-42.127929588347705, 61.000278252741154], [-86.17871099251512, 61.00027829214559], [-152.25488278218177, 61.000278219313], [-179.00057392307653, 61.000277909079024], [-179.00049757955398, -56.000277936790376], 

Composite

In [65]:
composite = ee.Image.cat([(sentinel),(evi),(ndvi),(ndbi),(dem)])

sentinel_vi = composite.clip(jedeb)

## Train the classifier 

In [88]:
bands = ['B2', 'B3', 'B4','B8', 'B8A', 'B11', 'B12',\
         'VVdec', 'VHdec', 'VVjan', 'VHjan', 'VVfeb', 'VHfeb', 'VVmar', 'VHmar', 'VVapr', 'VHapr',\
         'EVI','NDBI','elevation']
# This property the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = sentinel_vi.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

In [89]:
# Train a 50-tree gradient boosting classifier from the training sample.
classifier = ee.Classifier.smileRandomForest(128).train(sample, label, bands)

## Classify the Image

In [90]:
# Classify the image with the same bands used for training.
classified_RF3 = sentinel_vi.select(bands).classify(classifier)

## Accuracy Assessment

Training Accuarcy

In [69]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [70]:
#train_accuracy.accuracy().getInfo()

In [71]:
#train_accuracy.kappa().getInfo()

In [72]:
#train_accuracy.producersAccuracy().getInfo()

In [73]:
#train_accuracy.consumersAccuracy().getInfo()

Validation Accuarcy

In [91]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF3.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [92]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0],
 [0, 0, 63, 0, 3, 4, 0, 0],
 [0, 0, 1, 27, 1, 5, 0, 0],
 [0, 0, 0, 0, 44, 0, 0, 0],
 [0, 0, 1, 1, 1, 48, 0, 2],
 [0, 0, 0, 0, 0, 0, 12, 0],
 [0, 0, 0, 0, 0, 2, 0, 7]]

In [93]:
test_accuracy.accuracy().getInfo()

0.9066666666666666

In [95]:
test_accuracy.kappa().getInfo()

0.8806998939554612

In [78]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.9],
 [0.7647058823529411],
 [0.9545454545454546],
 [0.9245283018867925],
 [1],
 [0.7777777777777778]]

In [96]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  1,
  0.9692307692307692,
  0.9642857142857143,
  0.8979591836734694,
  0.8135593220338984,
  1,
  0.7777777777777778]]

# Visualize Results

In [97]:
# Define a palette for the classification.
Palette = [
    '419BDF', #water
    '397D49', #trees
    '88B053', #grass
    '6efc6a', #irrigation
    'E49635', #crops
    'C4281B', #built
    'A59B8F', #bare
]
Map.addLayer(classified_RF3, {'palette': Palette, 'min': 1, 'max': 7}, 'classification')
Map

Map(bottom=493351.0, center=[10.662294934332717, 37.80292510986329], controls=(WidgetControl(options=['positio…

# Calculate Statistics

In [81]:
# Calculate total area of a single class (eq(*))     * is the class number 
total_area = classified_RF1.multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

total_area.getInfo()

{'classification': 3383733707.7465363}

In [82]:
# Calculate total area of a single class (eq(*))     * is the class number 
area1 = classified_RF1.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

print(area1.getInfo(),'Area in square meter')

{'classification': 79709632.35718857} Area in square meter


In [83]:
# Calculate total area of a single class (eq(*))     * is the class number 
area2 = classified_RF2.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

print(area2.getInfo(),'Area in square meter')

{'classification': 55332138.34956268} Area in square meter


In [98]:
# Calculate total area of a single class (eq(*))     * is the class number 
area3 = classified_RF3.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

print(area3.getInfo(),'Area in square meter')

{'classification': 39248658.96113898} Area in square meter
